In [108]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense
from keras.utils import to_categorical 

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

df = pd.read_csv('datamahasiswa2015.csv')
df.head()

,No,NIM,OOP,RPB,RPL,PI,APSI,WEB,Statistik,Matdis,...,Desjar,KSI,Pilihan 1,Peminatan,MP1,MP2,MP3,Status,Tanggal Lulus,Keterangan
0,1,1202150001,2.0,3.5,0.0,3.5,3.0,3.0,2.0,2.0,...,3.5,2.0,TECHNO,TECHNO,3.5,3.0,3.0,GRADUATED,30/08/19,TEPAT WAKTU
1,2,1202150002,3.0,3.5,3.5,4.0,3.0,3.0,3.5,3.5,...,3.5,3.5,ERP,ERP,4.0,4.0,3.5,GRADUATED,10/07/19,TEPAT WAKTU
2,3,1202150003,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,OUT,OUT,0.0,0.0,0.0,RESIGN,OUT,TIDAK TEPAT WAKTU
3,4,1202150004,3.5,4.0,3.5,4.0,3.5,4.0,3.5,3.5,...,4.0,4.0,EAD,EAD,3.5,4.0,2.0,GRADUATED,05/07/19,TEPAT WAKTU
4,5,1202150005,2.0,2.5,2.0,1.0,3.5,1.0,2.0,1.0,...,2.0,1.0,OUT,OUT,0.0,0.0,0.0,STUDENT,OUT,TIDAK TEPAT WAKTU


In [109]:
new_class = [0] * len(df['Peminatan'])
for index, row in df.iterrows() :
    if row["Peminatan"] == "TECHNO" :
        new_class[index] = 1
    elif row['Peminatan'] == "EAD" :
        new_class[index] = 2
    elif row['Peminatan'] == "EDM" :
        new_class[index] = 3
    elif row['Peminatan'] == "ERP" :
        new_class[index] = 4
    elif row['Peminatan'] == "EA" :
        new_class[index] = 5
    elif row['Peminatan'] == "EIM" :
        new_class[index] = 6
    elif row['Peminatan'] == "ISM" :
        new_class[index] = 7
df["Peminatan"] = new_class
df.head()

,No,NIM,OOP,RPB,RPL,PI,APSI,WEB,Statistik,Matdis,...,Desjar,KSI,Pilihan 1,Peminatan,MP1,MP2,MP3,Status,Tanggal Lulus,Keterangan
0,1,1202150001,2.0,3.5,0.0,3.5,3.0,3.0,2.0,2.0,...,3.5,2.0,TECHNO,1,3.5,3.0,3.0,GRADUATED,30/08/19,TEPAT WAKTU
1,2,1202150002,3.0,3.5,3.5,4.0,3.0,3.0,3.5,3.5,...,3.5,3.5,ERP,4,4.0,4.0,3.5,GRADUATED,10/07/19,TEPAT WAKTU
2,3,1202150003,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,OUT,0,0.0,0.0,0.0,RESIGN,OUT,TIDAK TEPAT WAKTU
3,4,1202150004,3.5,4.0,3.5,4.0,3.5,4.0,3.5,3.5,...,4.0,4.0,EAD,2,3.5,4.0,2.0,GRADUATED,05/07/19,TEPAT WAKTU
4,5,1202150005,2.0,2.5,2.0,1.0,3.5,1.0,2.0,1.0,...,2.0,1.0,OUT,0,0.0,0.0,0.0,STUDENT,OUT,TIDAK TEPAT WAKTU


In [110]:
#change column in dataframe
dfNew = df[df.Peminatan != 0]
dfNew = dfNew.drop(['Status','Keterangan','Pilihan 1','Tanggal Lulus','No','NIM','MP1','MP2','MP3','RPL'],axis=1)
dfNew.rename(columns={'KSI':'Manprosi'}, inplace=True)
dfNew.head()

,OOP,RPB,PI,APSI,WEB,Statistik,Matdis,Alpro,Strukdat,SE,PO,SCM,EA,Basdat,Manjarkom,Sisop,MSDM,Desjar,Manprosi,Peminatan
0,2.0,3.5,3.5,3.0,3.0,2.0,2.0,3.0,2.0,3.5,3.5,3.0,3.5,2.0,2.0,2.5,3.5,3.5,2.0,1
1,3.0,3.5,4.0,3.0,3.0,3.5,3.5,3.5,3.0,3.5,4.0,4.0,3.0,3.5,2.5,3.5,3.5,3.5,3.5,4
3,3.5,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0,3.0,4.0,4.0,3.0,3.5,3.5,3.5,3.5,4.0,4.0,2
6,3.5,2.0,3.0,3.5,2.5,3.5,2.5,2.0,3.5,3.5,3.5,3.5,2.5,3.5,3.5,3.5,3.0,4.0,4.0,6
7,2.5,3.0,3.0,3.5,3.5,2.0,1.0,3.0,2.0,4.0,3.5,2.5,2.5,2.0,3.0,3.5,3.5,3.5,2.5,4


In [111]:
#select target column and predictor column
target = ["Peminatan"]
predictors = list(set(list(dfNew.columns))-set(target))
print(predictors)

['Statistik', 'Matdis', 'Manprosi', 'SE', 'MSDM', 'EA', 'Strukdat', 'SCM', 'OOP', 'PO', 'Sisop', 'PI', 'Alpro', 'RPB', 'WEB', 'Desjar', 'Basdat', 'APSI', 'Manjarkom']


In [112]:
#get predictor data
dfNew[predictors] = dfNew[predictors]/dfNew[predictors].max()
dfNew[predictors].head()

,Statistik,Matdis,Manprosi,SE,MSDM,EA,Strukdat,SCM,OOP,PO,Sisop,PI,Alpro,RPB,WEB,Desjar,Basdat,APSI,Manjarkom
0,0.500,0.500,0.500,0.875,0.875,0.875,0.500,0.750,0.500,0.875,0.625,0.875,0.750,0.875,0.750,0.875,0.500,0.750,0.500
1,0.875,0.875,0.875,0.875,0.875,0.750,0.750,1.000,0.750,1.000,0.875,1.000,0.875,0.875,0.750,0.875,0.875,0.750,0.625
3,0.875,0.875,1.000,0.750,0.875,0.750,1.000,1.000,0.875,1.000,0.875,1.000,0.875,1.000,1.000,1.000,0.875,0.875,0.875
6,0.875,0.625,1.000,0.875,0.750,0.625,0.875,0.875,0.875,0.875,0.875,0.750,0.500,0.500,0.625,1.000,0.875,0.875,0.875
7,0.500,0.250,0.625,1.000,0.875,0.625,0.500,0.625,0.625,0.875,0.875,0.750,0.750,0.750,0.875,0.875,0.500,0.875,0.750


In [113]:
X = dfNew[target]
y = dfNew[predictors]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=60)
print(X_train.shape)
print(y_train.shape)
print(X_test.head())
print(y_test.shape)

(188, 1)
(188, 19)
     Peminatan
223          6
253          5
297          4
278          5
266          5
(126, 19)


In [114]:
X_trains = to_categorical(X_train)
X_tests = to_categorical(X_test)

print(X_trains.shape)

(188, 8)


In [115]:
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=19))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(8, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [116]:
model.fit(y_train, X_trains, epochs=200)

Epoch 1/200
188/188 [==============================] - 0s 638us/step - loss: 2.0367 - accuracy: 0.1436
Epoch 2/200
188/188 [==============================] - 0s 43us/step - loss: 1.9242 - accuracy: 0.2128
Epoch 3/200
188/188 [==============================] - 0s 43us/step - loss: 1.8629 - accuracy: 0.2660
Epoch 4/200
188/188 [==============================] - 0s 42us/step - loss: 1.8445 - accuracy: 0.2660
Epoch 5/200
188/188 [==============================] - 0s 43us/step - loss: 1.8359 - accuracy: 0.2606
Epoch 6/200
188/188 [==============================] - 0s 85us/step - loss: 1.8278 - accuracy: 0.2500
Epoch 7/200
188/188 [==============================] - 0s 85us/step - loss: 1.8136 - accuracy: 0.2766
Epoch 8/200
188/188 [==============================] - 0s 85us/step - loss: 1.8127 - accuracy: 0.2660
Epoch 9/200
188/188 [==============================] - 0s 85us/step - loss: 1.7974 - accuracy: 0.2926
Epoch 10/200
188/188 [==============================] - 0s 85us/step - loss: 1.79

In [117]:
pred_train= model.predict(y_train)
scores = model.evaluate(y_train, X_trains, verbose=0)
print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores[1], 1 - scores[1]))   
 
pred_test= model.predict(y_test)
scores2 = model.evaluate(y_test, X_tests, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))   

Accuracy on training data: 0.6808510422706604% 
 Error on training data: 0.3191489577293396
Accuracy on test data: 0.4126984179019928% 
 Error on test data: 0.5873015820980072


In [118]:
dataTesting = pd.read_excel('Nilai Master Pipe.xlsx')
dataTesting.head()

,nim,nama,OOP,RPB,PI,APSI,WEB,Statistik,Matdis,Alpro,...,SE,PO,SCM,EA,Basdat,Manjarkom,Sisop,MSDM,Desjar,Manprosi
0,1202170006,SHINTA EKA RISA SINURAYA,4.0,3.5,2.0,2.0,3.5,4.0,2.5,3.5,...,4.0,3.0,3.0,3.0,3.0,3.0,3.0,3.5,3.5,2.5
1,1202170010,COKORDA AGUNG JAYA LAKSMANA PRAKASA,4.0,3.5,4.0,2.5,4.0,3.0,4.0,3.0,...,4.0,3.5,3.0,3.5,3.5,3.5,3.0,3.5,3.5,3.0
2,1202170011,RAFIF FAKHRI RAMADHAN,4.0,3.0,3.5,2.0,3.5,3.5,3.5,3.5,...,4.0,3.5,3.5,3.5,4.0,3.5,3.0,4.0,3.5,3.0
3,1202170015,M IKHSAN PRATAMA P,4.0,3.5,3.5,3.5,3.5,3.5,3.5,4.0,...,3.5,2.5,3.5,3.0,3.5,3.0,3.5,3.5,3.5,2.5
4,1202170018,DINDA SONIA TIARA,2.0,2.5,3.0,3.5,3.5,2.5,2.5,3.0,...,3.5,3.5,2.5,3.5,3.0,2.5,3.0,3.5,2.0,3.5


In [120]:
dataTestingFilter = dataTesting[predictors]
dataTestingHasil = model.predict_classes(dataTestingFilter)

dataTesting["Peminatan"] = dataTestingHasil
dataTesting.rename(columns={'nim':'NIM'}, inplace=True)
dataTesting.drop(columns="nama",axis=1).head()

,NIM,OOP,RPB,PI,APSI,WEB,Statistik,Matdis,Alpro,Strukdat,...,PO,SCM,EA,Basdat,Manjarkom,Sisop,MSDM,Desjar,Manprosi,Peminatan
0,1202170006,4.0,3.5,2.0,2.0,3.5,4.0,2.5,3.5,2.5,...,3.0,3.0,3.0,3.0,3.0,3.0,3.5,3.5,2.5,3
1,1202170010,4.0,3.5,4.0,2.5,4.0,3.0,4.0,3.0,4.0,...,3.5,3.0,3.5,3.5,3.5,3.0,3.5,3.5,3.0,2
2,1202170011,4.0,3.0,3.5,2.0,3.5,3.5,3.5,3.5,4.0,...,3.5,3.5,3.5,4.0,3.5,3.0,4.0,3.5,3.0,2
3,1202170015,4.0,3.5,3.5,3.5,3.5,3.5,3.5,4.0,4.0,...,2.5,3.5,3.0,3.5,3.0,3.5,3.5,3.5,2.5,1
4,1202170018,2.0,2.5,3.0,3.5,3.5,2.5,2.5,3.0,3.5,...,3.5,2.5,3.5,3.0,2.5,3.0,3.5,2.0,3.5,4


In [121]:
pred_test= model.predict(dataTestingFilter)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))

Accuracy on test data: 0.4126984179019928% 
 Error on test data: 0.5873015820980072


In [122]:
new_class = [0] * len(dataTesting['Peminatan'])
for index, row in dataTesting.iterrows() :
    if row["Peminatan"] == 1 :
        new_class[index] = "TECHNO"
    elif row['Peminatan'] == 2 :
        new_class[index] = "EAD"
    elif row['Peminatan'] == 3 :
        new_class[index] = "EDM"
    elif row['Peminatan'] == 4 :
        new_class[index] = "ERP"
    elif row['Peminatan'] == 5 :
        new_class[index] = "EA"
    elif row['Peminatan'] == 6 :
        new_class[index] = "EIM"
    elif row['Peminatan'] == 7 :
        new_class[index] = "ISM"
dataTesting["Peminatan"] = new_class
dataTesting.head()

,NIM,nama,OOP,RPB,PI,APSI,WEB,Statistik,Matdis,Alpro,...,PO,SCM,EA,Basdat,Manjarkom,Sisop,MSDM,Desjar,Manprosi,Peminatan
0,1202170006,SHINTA EKA RISA SINURAYA,4.0,3.5,2.0,2.0,3.5,4.0,2.5,3.5,...,3.0,3.0,3.0,3.0,3.0,3.0,3.5,3.5,2.5,EDM
1,1202170010,COKORDA AGUNG JAYA LAKSMANA PRAKASA,4.0,3.5,4.0,2.5,4.0,3.0,4.0,3.0,...,3.5,3.0,3.5,3.5,3.5,3.0,3.5,3.5,3.0,EAD
2,1202170011,RAFIF FAKHRI RAMADHAN,4.0,3.0,3.5,2.0,3.5,3.5,3.5,3.5,...,3.5,3.5,3.5,4.0,3.5,3.0,4.0,3.5,3.0,EAD
3,1202170015,M IKHSAN PRATAMA P,4.0,3.5,3.5,3.5,3.5,3.5,3.5,4.0,...,2.5,3.5,3.0,3.5,3.0,3.5,3.5,3.5,2.5,TECHNO
4,1202170018,DINDA SONIA TIARA,2.0,2.5,3.0,3.5,3.5,2.5,2.5,3.0,...,3.5,2.5,3.5,3.0,2.5,3.0,3.5,2.0,3.5,ERP


In [123]:
dataTesting.to_csv('seleksi-peminatan-cnn.csv')